# 1. Data Preprocessing

In [13]:
# Import libraries
import numpy as np
import pandas as pd 
from ortools.sat.python import cp_model

In [2]:
# Read Data 
def read_data(file):
    with open(file, 'r') as file:
        # read first line 
        n, d, a, b = [int(x) for x in file.readline().split()] 

        # Matrix (n+1,d+1) full 0, if staff i rest day d(i) -> convert to 1 
        F = np.full((n+1, d+1), 0)  
        for staff in range(n):
            # read each line to end, [:-1] bcs end of each line is -1 
            temp = [int(x) for x in file.readline().split()[:-1]]
            for day in temp:
                F[staff, day-1] = 1
    return n, d, a, b, F

# Input 
n, d, a, b, F = read_data("data_18.txt")

In [12]:
print("Number of Staffs: {}\nNumber of Days: {}\nRange staffs of a shift: ({},{})\nList rest day of staff i:\n{}".format(n,d,a,b,F))

Number of Staffs: 9
Number of Days: 5
Range staffs of a shift: (1,3)
List rest day of staff i:
[[0 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


# 2. CSOP


In [ ]:
# Declare variables
model = cp_model.CpModel()

X = {}
for staff in range(n):
    for day in range(d):
        for kip in range(1,5):
            X[staff, day, kip] = model.NewIntVar(0, 1, f"x[{staff},{day},{kip}]")

''' X[person,day,kip] 
-> 0 nếu ngày day kíp kip người person không làm
 -> 1 nếu ngày day kíp kip người person làm '''

# Each day, an employee can only work one shift at most
# If you work the night shift the day before, you can rest the next day
for staff in range(n):
    for day in range(d):
        if F[staff, day] == 0:
            if day == 0:
                model.Add(sum([X[staff, day, kip]
                                for kip in range(1,5)]) == 1)
            else:
                model.Add(sum([X[staff, day, kip]
                                for kip in range(1,5)]) + X[staff, day - 1, 4] == 1)
        else:
            model.Add(sum([X[staff, day, kip] for kip in range(1,5)]) == 0)

# Each shift in each day has at least [a] employees and at most [b] employees
for day in range(d):
    for kip in range(1,5):
        model.Add(sum([X[person, day, kip] for person in range(n)]) <= b)
        model.Add(sum([X[person, day, kip] for person in range(n)]) >= a)

# F(i): list of employee rest days i
# The maximum number of night shifts assigned to a specific employee is the smallest
max_ca_dem = model.NewIntVar(1, int(d/2) + 1, 'max_ca_dem')
for staff in range(n):
    model.Add(sum([X[staff, day, 4]
                    for day in range(d)]) - max_ca_dem <= 0)

# Objective Function
model.Minimize(max_ca_dem)

# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
  print("Min: ", solver.ObjectiveValue())

In [ ]:
# Print Solver
if status == cp_model.OPTIMAL:
  print("Min: ", solver.ObjectiveValue())